In [2]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math
import numpy as np
import re
from shapely.geometry import Polygon, LineString, Point
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import CocoDetection
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision.transforms as T
from torch.optim import SGD, Adam, Adadelta
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision import transforms
from torch.utils.data._utils.collate import default_collate
import torchvision
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.transforms import functional as F
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import random
from math import radians, cos, sin
import ast
import shutil

# 1. Unzip & Prepare Directory

In [ ]:
import tarfile

tar_path = './ds2_dense.tar.gz'
extract_path = './ds2_dense/'

with tarfile.open(tar_path, "r:gz") as tar:
    tar.extractall(path=extract_path)

# fix nested folder
nested = os.path.join(extract_path, 'ds2_dense')
if os.path.exists(nested):
    for item in os.listdir(nested):
        shutil.move(os.path.join(nested, item), extract_path)
    shutil.rmtree(nested)

# 2. Load Json file

In [3]:
# 加载 JSON
with open('./ds2_dense/deepscores_train.json') as f:
    data1 = json.load(f)
with open('./ds2_dense/deepscores_test.json') as f:
    data2 = json.load(f)

# 提取图像和标注
train_images = pd.DataFrame(data1['images'])
train_annots = pd.DataFrame(data1['annotations']).T

test_images = pd.DataFrame(data2['images'])
test_annots = pd.DataFrame(data2['annotations']).T

# 3. Splitting the Images into Train and Test sets

In [ ]:
image_dir = './ds2_dense/images'
train_dir = './ds2_dense/images/train'
test_dir = './ds2_dense/images/test'
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

In [ ]:
# Move train images to train directory
for image_filename in train_images['filename']:
   src_path = os.path.join(image_dir, image_filename)
   dest_path = os.path.join(train_dir, image_filename)
   shutil.move(src_path, dest_path)

In [ ]:
# Move train images to test directory
for image_filename in test_images['filename']:
   src_path = os.path.join(image_dir, image_filename)
   dest_path = os.path.join(test_dir, image_filename)
   shutil.move(src_path, dest_path)

# 4. Generate Label Mapping (deepscores only)
The yolo model wants labels to be zero-based indexing


In [4]:
label_dict = list(data1.values())[2]

# 先过滤出 annotation_set == 'deepscores' 的项
filtered_items = [(k, v) for k, v in label_dict.items() if v['annotation_set'] == 'deepscores']

# 用 enumerate 连续编号
cat_dict = {
    k: {
        'old_id': int(k),
        'name': v['name'],
        'label': i
    } for i, (k, v) in enumerate(filtered_items)
}

# 转为 DataFrame
df_labels = pd.DataFrame.from_dict(cat_dict, orient='index').reset_index(drop=True)

df_labels


,old_id,name,label
0,1,brace,0
1,2,ledgerLine,1
2,3,repeatDot,2
3,4,segno,3
4,5,coda,4
...,...,...,...
131,132,tuplet8,131
132,133,tuplet9,132
133,134,tupletBracket,133
134,135,staff,134


# 5. Attach Label to each obbox

In [5]:
class_mapping = dict(zip(df_labels['old_id'].astype(str), df_labels['label']))

def map_label(cat_ids):
    return max([class_mapping.get(str(cid)) for cid in cat_ids if str(cid) in class_mapping], default=None)

train_annots['label'] = train_annots['cat_id'].apply(map_label)
test_annots['label'] = test_annots['cat_id'].apply(map_label)

train_annots.head(10)

,a_bbox,o_bbox,cat_id,area,img_id,comments,label
1020,"[116.0, 139.0, 2315.0, 206.0]","[2315.0, 206.0, 2315.0, 139.0, 116.0, 139.0, 1...","[135, 208]",18945,679,instance:#000010;,134
1021,"[116.0, 309.0, 2315.0, 376.0]","[2315.0, 376.0, 2315.0, 309.0, 116.0, 309.0, 1...","[135, 208]",19223,679,instance:#000021;,134
1022,"[1880.0, 561.0, 1911.0, 564.0]","[1911.0, 564.0, 1911.0, 561.0, 1880.0, 561.0, ...","[2, 138]",120,679,instance:#000022;,1
1023,"[1883.0, 578.0, 1911.0, 580.0]","[1911.0, 580.0, 1911.0, 578.0, 1883.0, 578.0, ...","[2, 138]",27,679,instance:#000023;,1
1024,"[1827.0, 561.0, 1857.0, 564.0]","[1857.0, 564.0, 1857.0, 561.0, 1827.0, 561.0, ...","[2, 138]",112,679,instance:#000024;,1
1025,"[1827.0, 578.0, 1857.0, 580.0]","[1857.0, 580.0, 1857.0, 578.0, 1827.0, 578.0, ...","[2, 138]",32,679,instance:#000025;,1
1026,"[1773.0, 561.0, 1804.0, 564.0]","[1804.0, 564.0, 1804.0, 561.0, 1773.0, 561.0, ...","[2, 138]",120,679,instance:#000026;,1
1027,"[1773.0, 578.0, 1804.0, 580.0]","[1804.0, 580.0, 1804.0, 578.0, 1773.0, 578.0, ...","[2, 138]",71,679,instance:#000027;,1
1028,"[1718.0, 561.0, 1748.0, 564.0]","[1748.0, 564.0, 1748.0, 561.0, 1718.0, 561.0, ...","[2, 138]",112,679,instance:#000028;,1
1029,"[1718.0, 578.0, 1748.0, 580.0]","[1748.0, 580.0, 1748.0, 578.0, 1718.0, 578.0, ...","[2, 138]",84,679,instance:#000029;,1


# 6. Adjust bounding box for 0 valued w/h

In [8]:
def adjust_bbox(bbox):
    x_min, y_min, x_max, y_max = bbox
    if x_min == x_max:
        x_min -= 1
        x_max += 1
    if y_min == y_max:
        y_min -= 1
        y_max += 1
    return [x_min, y_min, x_max, y_max]

train_annots['bbox'] = train_annots['a_bbox'].apply(adjust_bbox)
test_annots['bbox'] = test_annots['a_bbox'].apply(adjust_bbox)

train_images.head(10)


,id,filename,width,height,ann_ids
0,2,lg-94161796-aug-gonville--page-3.png,1960,2772,"[1113019, 1113020, 1113021, 1113022, 1113023, ..."
1,3,lg-139957803-aug-emmentaler--page-5.png,1960,2772,"[182814, 182815, 182816, 182817, 182818, 18281..."
2,4,lg-63506682-aug-emmentaler--page-6.png,1960,2772,"[435136, 435137, 435138, 435139, 435140, 43514..."
3,7,lg-102548668-aug-gonville--page-4.png,1960,2772,"[998808, 998809, 998810, 998811, 998812, 99881..."
4,8,lg-233786100286899765-aug-gutenberg1939--page-...,2970,4201,"[774295, 774296, 774297, 774298, 774299, 77430..."
5,9,lg-26406557-aug-lilyjazz--page-1.png,1960,2772,"[473724, 473725, 473726, 473727, 473728, 47372..."
6,10,lg-695667819306673755-aug-lilyjazz-.png,1960,2772,"[1032754, 1032755, 1032756, 1032757, 1032758, ..."
7,11,lg-156284447442202986-aug-gutenberg1939--page-...,1960,2772,"[53456, 53457, 53458, 53459, 53460, 53461, 534..."
8,12,lg-101766503886095953-aug-gonville--page-4.png,1960,2772,"[278463, 278464, 278465, 278466, 278467, 27846..."
9,13,lg-69678954220027474-aug-beethoven--page-176.png,1960,2772,"[219162, 219163, 219164, 219165, 219166, 21916..."


# 7. Merge Annotation and Image df

In [9]:
train_images.rename(columns={'id': 'img_id'}, inplace=True)
test_images.rename(columns={'id': 'img_id'}, inplace=True)

train_annots['img_id'] = train_annots['img_id'].astype(str)
test_annots['img_id'] = test_annots['img_id'].astype(str)

train_images['img_id'] = train_images['img_id'].astype(str)
test_images['img_id'] = test_images['img_id'].astype(str)

train_df = pd.merge(train_annots, train_images, on='img_id')
test_df = pd.merge(test_annots, test_images, on='img_id')

train_df.head(10)

,a_bbox,o_bbox,cat_id,area,img_id,comments,label,bbox,filename,width,height,ann_ids
0,"[116.0, 139.0, 2315.0, 206.0]","[2315.0, 206.0, 2315.0, 139.0, 116.0, 139.0, 1...","[135, 208]",18945,679,instance:#000010;,134,"[116.0, 139.0, 2315.0, 206.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102..."
1,"[116.0, 309.0, 2315.0, 376.0]","[2315.0, 376.0, 2315.0, 309.0, 116.0, 309.0, 1...","[135, 208]",19223,679,instance:#000021;,134,"[116.0, 309.0, 2315.0, 376.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102..."
2,"[1880.0, 561.0, 1911.0, 564.0]","[1911.0, 564.0, 1911.0, 561.0, 1880.0, 561.0, ...","[2, 138]",120,679,instance:#000022;,1,"[1880.0, 561.0, 1911.0, 564.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102..."
3,"[1883.0, 578.0, 1911.0, 580.0]","[1911.0, 580.0, 1911.0, 578.0, 1883.0, 578.0, ...","[2, 138]",27,679,instance:#000023;,1,"[1883.0, 578.0, 1911.0, 580.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102..."
4,"[1827.0, 561.0, 1857.0, 564.0]","[1857.0, 564.0, 1857.0, 561.0, 1827.0, 561.0, ...","[2, 138]",112,679,instance:#000024;,1,"[1827.0, 561.0, 1857.0, 564.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102..."
5,"[1827.0, 578.0, 1857.0, 580.0]","[1857.0, 580.0, 1857.0, 578.0, 1827.0, 578.0, ...","[2, 138]",32,679,instance:#000025;,1,"[1827.0, 578.0, 1857.0, 580.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102..."
6,"[1773.0, 561.0, 1804.0, 564.0]","[1804.0, 564.0, 1804.0, 561.0, 1773.0, 561.0, ...","[2, 138]",120,679,instance:#000026;,1,"[1773.0, 561.0, 1804.0, 564.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102..."
7,"[1773.0, 578.0, 1804.0, 580.0]","[1804.0, 580.0, 1804.0, 578.0, 1773.0, 578.0, ...","[2, 138]",71,679,instance:#000027;,1,"[1773.0, 578.0, 1804.0, 580.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102..."
8,"[1718.0, 561.0, 1748.0, 564.0]","[1748.0, 564.0, 1748.0, 561.0, 1718.0, 561.0, ...","[2, 138]",112,679,instance:#000028;,1,"[1718.0, 561.0, 1748.0, 564.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102..."
9,"[1718.0, 578.0, 1748.0, 580.0]","[1748.0, 580.0, 1748.0, 578.0, 1718.0, 578.0, ...","[2, 138]",84,679,instance:#000029;,1,"[1718.0, 578.0, 1748.0, 580.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102..."


# 8. Convert BBOX to YOLO format

In [ ]:
def convert_to_yolo(bbox, width, height):
    x_min, y_min, x_max, y_max = bbox
    x_center = (x_min + x_max) / 2 / width
    y_center = (y_min + y_max) / 2 / height
    w = (x_max - x_min) / width
    h = (y_max - y_min) / height
    return [x_center, y_center, w, h]

train_df['yolo_bbox'] = train_df.apply(lambda r: convert_to_yolo(r['bbox'], r['width'], r['height']), axis=1)
test_df['yolo_bbox'] = test_df.apply(lambda r: convert_to_yolo(r['bbox'], r['width'], r['height']), axis=1)
train_df.head(10)


# 9. Output YOLO training information as txt

In [ ]:
import os

def export_yolo_txt(df, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for fname, group in df.groupby('filename'):
        txt_path = os.path.join(output_dir, os.path.splitext(fname)[0] + '.txt')
        with open(txt_path, 'w') as f:
            for _, row in group.iterrows():
                cls = row['label']
                x, y, w, h = row['yolo_bbox']
                f.write(f"{cls} {x} {y} {w} {h}\n")

export_yolo_txt(train_df, './ds2_dense/labels/train')
export_yolo_txt(test_df, './ds2_dense/labels/test')

# 10. Save Yolo Training Meta information

In [ ]:
import yaml

yaml_dict = {
    'path': os.path.abspath('./ds2_dense'),
    'train': 'images/train',
    'val': 'images/test',
    'names': df_labels.set_index('label')['name'].to_dict()
}

with open('deep_scores.yaml', 'w') as f:
    yaml.dump(yaml_dict, f)

# 11. Train Yolo Model

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8m.pt') 


In [ ]:
# Check if CUDA is available and configure training accordingly

device = torch.device("cuda:0")

if torch.cuda.is_available():
    print('Training with GPU.')
    results = model.train(data="deep_scores.yaml", epochs=10, batch=3, imgsz=[704, 992], device=device, patience=0, deterministic=True, amp=True) # rect=True, ,[928, 1312], imgsz=[704, 992], imgsz=[1960, 2772]
    
else:
    results = model.train(data="deep_scores.yaml", imgsz=640, rect=True, epochs = 10, batch=4)  # patience=100, device="mps", deterministic=True, amp=True